In [1]:
import os
import itertools
from attrs import define
from codes.train import train
# from codes.train_dist import train
from codes.optimizers.base import Optimizer
# from code.problems import Problem
from codes import Loss
from codes.datasets import Dataset
from codes.models import Model

# %matplotlib widget
%load_ext autoreload
%autoreload 2

/home/nazya/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def zip_dict(**kwargs):
    keys = kwargs.keys()
    for instance in zip(*kwargs.values()):
        yield dict(zip(keys, instance))


def product_dict(**kwargs):
    keys = kwargs.keys()
    for instance in itertools.product(*kwargs.values()):
        yield dict(zip(keys, instance))

In [3]:
os.environ["MKL_THREADING_LAYER"] = "AMD"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["TORCH_DEVICE"] = "cuda"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [4]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_CHECK_EXIST'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd())

# 3 classes emotion

In [5]:
@define
class BaseConfig():
    nepochs:         int = 40
    # niters:          int = 2500
    npeers:          int = 20
    seed:            int = 0

    loss:           Loss = Loss.CrossEntropyLoss
    model:         Model = Model.BERT
    dataset:     Dataset = Dataset.GoEmotions

    # nsamples:        int = 1000
    valenabled_:    bool = False
    valnsamples_:    int = None
    nclasses:        int = 28
    hratio:        float = None

    optimizer: Optimizer = None
    batchsize:       int = 40
    lr:            float = 1e-2

    trueweights:    bool = None

    mdbatchsize_:    int = 30
    mdniters_:       int = None
    mdlr_:           int = None
    mdfull_:        bool = None

In [6]:
os.environ['MLFLOW_RUN_TAGS'] = str(dict(about=f'{BaseConfig().valenabled_=} / {BaseConfig().nclasses} / single thread'))

args_grid = dict(
    # hratio=[0.99, 0.9, 0.7, 0.5],
    hratio=[0.9],
    mdlr_=[0.1],
    seed=[0,1,2],
    # valenabled=[True, False]
    valenabled_=[False]
)

for d in product_dict(**args_grid):

    os.environ['MLFLOW_RUN_NAME'] = 'SGD Full'
    config = BaseConfig(**d)
    config.mdlr_ = None
    config.mdbatchsize_ = None
    config.optimizer = Optimizer.SGD
    config.trueweights = False
    # %time train(config)
    %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'SGD Ideal'
    config = BaseConfig(**d)
    config.mdlr_ = None
    config.mdbatchsize_ = None
    config.hratio = None
    config.optimizer = Optimizer.SGD
    config.trueweights = True
    %time train(config)

    # os.environ['MLFLOW_RUN_NAME'] = 'MeritFed MD'
    # config = BaseConfig(**d)
    # config.optimizer = Optimizer.MeritFed
    # config.mdfull_ = True
    # config.mdniters_ = 1
    # # config.md_lr_ = 0.05
    # %time train(config)

    # os.environ['MLFLOW_RUN_NAME'] = 'MeritFed SMD'
    # config = BaseConfig(**d)
    # config.optimizer = Optimizer.MeritFed
    # config.mdfull_ = False
    # config.mdniters_ = 10
    # # config.md_lr_ = 0.05
    # %time train(config)

    config = BaseConfig(**d)
    config.optimizer = Optimizer.TAWT
    os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    config.mdlr_ = None
    %time train(config)

    config = BaseConfig(**d)
    config.optimizer = Optimizer.FedAdp
    os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    config.mdlr_ = None
    %time train(config)

CPU times: user 1.57 s, sys: 536 ms, total: 2.11 s
Wall time: 2.11 s
CPU times: user 1.51 s, sys: 688 ms, total: 2.2 s
Wall time: 2.2 s
CPU times: user 1.85 s, sys: 600 ms, total: 2.45 s
Wall time: 2.46 s
CPU times: user 1.7 s, sys: 612 ms, total: 2.31 s
Wall time: 2.32 s
CPU times: user 1.77 s, sys: 708 ms, total: 2.48 s
Wall time: 2.48 s
CPU times: user 1.53 s, sys: 711 ms, total: 2.24 s
Wall time: 2.25 s
CPU times: user 1.83 s, sys: 848 ms, total: 2.68 s
Wall time: 2.72 s
CPU times: user 1.61 s, sys: 648 ms, total: 2.26 s
Wall time: 2.57 s
CPU times: user 2.02 s, sys: 955 ms, total: 2.98 s
Wall time: 3.15 s
CPU times: user 1.69 s, sys: 671 ms, total: 2.36 s
Wall time: 2.98 s
trueweights=[1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Step 1000: train-loss: 0.00780 train-accuracy: 99.80000 test-loss: 1.59432 test-accuracy: 72.80100
CPU times: user 30min 19s, sys: 49min 28s, total: 1h 19min 48s
Wall time: 1h 19min 55s
trueweights=[1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step 1000: train-loss: 0.01725 train-accuracy: 99.70000 test-loss: 1.21586 test-accuracy: 73.923891
CPU times: user 32min 34s, sys: 50min 13s, total: 1h 22min 47s
Wall time: 1h 22min 38s
